In [2]:
from google.colab import drive
drive.mount('/content/drive')

from pathlib import Path

project_folder = Path('/content/drive/My Drive/cs229-audio-project/fine_tuning_llm/')

Mounted at /content/drive


In [3]:
import pandas as pd

data_input = pd.read_csv(project_folder / 'data_triplets_after_llm_latex_2024-03-16_02-43-06.csv')
display(data_input)

,Spoken Text,Formula ID,Latex,Audio Path,Spoken Text Version,Transcribed Text,LLM Predicted Latex
0,The rank of a matrix A.,1,\text{{Rank}}(A),/content/drive/My Drive/cs229-audio-project/au...,1,The rank of a matrix A,\text{rank}(A)
1,The nullity of a given matrix A.,2,\text{{Nullity}}(A),/content/drive/My Drive/cs229-audio-project/au...,1,the nullity of a given matrix A.,\text{Nullity}(A)
2,P of z equals three hundred fourteen plus one ...,3,p(z) = 314 + 122v + 355b^2 + \cdots + 444b^n,/content/drive/My Drive/cs229-audio-project/au...,1,p of variable equals number plus number times ...,p(x) = a_0 + a_1x + a_2x^2 + \ldots + a_nx^n
3,Force vector equals mass times acceleration ve...,4,\vec{F} = m\vec{\alpha},/content/drive/My Drive/cs229-audio-project/au...,1,Force vector equals mass times acceleration ve...,\vec{F} = m \vec{a}
4,Energy equals mass times the speed of light sq...,5,E = m{c}^2,/content/drive/My Drive/cs229-audio-project/au...,1,E equals mass times the speed of light squared,E = m \cdot c^2
...,...,...,...,...,...,...,...
2461,The cross product of the vectors with componen...,818,"[106,800,213] \times [450,206,869]",/content/drive/My Drive/cs229-audio-project/au...,3,The cross product of the vectors with componen...,\begin{equation}\n\mathbf{v_1} = \begin{bmatri...
2462,The cross product of the vectors with componen...,819,"[890,879,829] \times [3381,338,180]",/content/drive/My Drive/cs229-audio-project/au...,3,The cross product of the vectors with componen...,\mathbf{v} = \begin{pmatrix} 890 \\ 879 \\ 829...
2463,The cross product of the vectors with componen...,820,"[336,739,485] \times [160,546,86]",/content/drive/My Drive/cs229-audio-project/au...,3,The cross product of the vectors with componen...,\begin{equation}\n\begin{pmatrix}\n336 \\\n739...
2464,The cross product of the vectors with componen...,821,"[52,837,748] \times [845,748,168]",/content/drive/My Drive/cs229-audio-project/au...,3,The cross product of the vectors with componen...,\[\n\begin{vmatrix}\n\mathbf{i} & \mathbf{j} &...


In [5]:
from datetime import datetime

def backup_df(df, prefix, folder_path):
    current_time = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")

    file_path = folder_path / f"{prefix}_{current_time}.csv"
    df.to_csv(file_path, index=False)

    print(f"DataFrame saved to: {file_path}")

In [14]:
!pip install tiktoken

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 27.3 MB/s eta 0:00:00


In [154]:
def make_message(english_of_equation, desired_latex_response):

    # purposely missing space since that's the way the inference is doing it (small bug)
    preprompt = "take this transcription and write the equation in latex.Don't allow any english and use only latex."
    return {
        "messages": [
            {"role": "system", "content" : preprompt},
            {"role": "user", "content" : english_of_equation},
            {"role": "assistant", "content": desired_latex_response}
        ]
    }

def make_messages(triplets_df):
    return triplets_df.apply(lambda row: make_message(row["Spoken Text"], row["Latex"]), axis=1)

from sklearn.model_selection import train_test_split

def train_test_dataset_from_input_data(triplets_df):
    """
    opein ai example:

    {"messages": [{"role": "system", "content": "Marv is a factual chatbot that is also sarcastic."}, {"role": "user", "content": "What's the capital of France?"}, {"role": "assistant", "content": "Paris, as if everyone doesn't know that already."}]}
    {"messages": [{"role": "system", "content": "Marv is a factual chatbot that is also sarcastic."}, {"role": "user", "content": "Who wrote 'Romeo and Juliet'?"}, {"role": "assistant", "content": "Oh, just some guy named William Shakespeare. Ever heard of him?"}]}
    {"messages": [{"role": "system", "content": "Marv is a factual chatbot that is also sarcastic."}, {"role": "user", "content": "How far is the Moon from Earth?"}, {"role": "assistant", "content": "Around 384,400 kilometers. Give or take a few, like that really matters."}]}
    """
    # train_df, test_df = train_test_split(triplets_df, test_size=test_ratio, random_state=42)
    train_size = 50
    valid_size = 25
    train_df, valid_df, test_df = triplets_df[:train_size], triplets_df[train_size:train_size+valid_size], triplets_df[train_size+valid_size:]


    training_dataset = list(make_messages(train_df).to_dict().values())
    test_dataset = list(make_messages(test_df).to_dict().values())
    valid_dataset = list(make_messages(valid_df).to_dict().values())
    return training_dataset, valid_dataset, test_dataset

In [156]:
print("total=", len(data_input))
data_input = data_input.dropna()
train, valid, test = train_test_dataset_from_input_data(data_input)
print("train=", len(train))
print("valid=", len(valid))
print("test=", len(test))

total= 2461
train= 50
valid= 25
test= 2386


In [158]:
import json
import tiktoken # for token counting
import numpy as np
from collections import defaultdict

for to_verify in (("train", train), ("test", test), ("valid", valid)):
    name, dataset = to_verify
    print("dataset name =", name, "-"*100)

    # Format error checks
    format_errors = defaultdict(int)

    for ex in dataset:
        if not isinstance(ex, dict):
            format_errors["data_type"] += 1
            continue

        messages = ex.get("messages", None)
        if not messages:
            format_errors["missing_messages_list"] += 1
            continue

        for message in messages:
            if "role" not in message or "content" not in message:
                format_errors["message_missing_key"] += 1

            if any(k not in ("role", "content", "name", "function_call", "weight") for k in message):
                format_errors["message_unrecognized_key"] += 1

            if message.get("role", None) not in ("system", "user", "assistant", "function"):
                format_errors["unrecognized_role"] += 1

            content = message.get("content", None)
            function_call = message.get("function_call", None)

            if (not content and not function_call) or not isinstance(content, str):
                format_errors["missing_content"] += 1

        if not any(message.get("role", None) == "assistant" for message in messages):
            format_errors["example_missing_assistant_message"] += 1

    if format_errors:
        print("Found errors:")
        for k, v in format_errors.items():
            print(f"{k}: {v}")
    else:
        print("No errors found")

    encoding = tiktoken.get_encoding("cl100k_base")

    # not exact!
    # simplified from https://github.com/openai/openai-cookbook/blob/main/examples/How_to_count_tokens_with_tiktoken.ipynb
    def num_tokens_from_messages(messages, tokens_per_message=3, tokens_per_name=1):
        num_tokens = 0
        for message in messages:
            num_tokens += tokens_per_message
            for key, value in message.items():
                num_tokens += len(encoding.encode(value))
                if key == "name":
                    num_tokens += tokens_per_name
        num_tokens += 3
        return num_tokens

    def num_assistant_tokens_from_messages(messages):
        num_tokens = 0
        for message in messages:
            if message["role"] == "assistant":
                num_tokens += len(encoding.encode(message["content"]))
        return num_tokens

    def print_distribution(values, name):
        print(f"\n#### Distribution of {name}:")
        print(f"min / max: {min(values)}, {max(values)}")
        print(f"mean / median: {np.mean(values)}, {np.median(values)}")
        print(f"p5 / p95: {np.quantile(values, 0.1)}, {np.quantile(values, 0.9)}")

    # Warnings and tokens counts
    n_missing_system = 0
    n_missing_user = 0
    n_messages = []
    convo_lens = []
    assistant_message_lens = []

    for ex in dataset:
        messages = ex["messages"]
        if not any(message["role"] == "system" for message in messages):
            n_missing_system += 1
        if not any(message["role"] == "user" for message in messages):
            n_missing_user += 1
        n_messages.append(len(messages))
        convo_lens.append(num_tokens_from_messages(messages))
        assistant_message_lens.append(num_assistant_tokens_from_messages(messages))

    print("Num examples missing system message:", n_missing_system)
    print("Num examples missing user message:", n_missing_user)
    print_distribution(n_messages, "num_messages_per_example")
    print_distribution(convo_lens, "num_total_tokens_per_example")
    print_distribution(assistant_message_lens, "num_assistant_tokens_per_example")
    n_too_long = sum(l > 4096 for l in convo_lens)
    print(f"\n{n_too_long} examples may be over the 4096 token limit, they will be truncated during fine-tuning")

    # Pricing and default n_epochs estimate
    MAX_TOKENS_PER_EXAMPLE = 4096

    TARGET_EPOCHS = 3
    MIN_TARGET_EXAMPLES = 100
    MAX_TARGET_EXAMPLES = 25000
    MIN_DEFAULT_EPOCHS = 1
    MAX_DEFAULT_EPOCHS = 25

    n_epochs = TARGET_EPOCHS
    n_train_examples = len(dataset)
    if n_train_examples * TARGET_EPOCHS < MIN_TARGET_EXAMPLES:
        n_epochs = min(MAX_DEFAULT_EPOCHS, MIN_TARGET_EXAMPLES // n_train_examples)
    elif n_train_examples * TARGET_EPOCHS > MAX_TARGET_EXAMPLES:
        n_epochs = max(MIN_DEFAULT_EPOCHS, MAX_TARGET_EXAMPLES // n_train_examples)

    n_billing_tokens_in_dataset = sum(min(MAX_TOKENS_PER_EXAMPLE, length) for length in convo_lens)
    print(f"Dataset has ~{n_billing_tokens_in_dataset} tokens that will be charged for during training")
    print(f"By default, you'll train for {n_epochs} epochs on this dataset")
    print(f"By default, you'll be charged for ~{n_epochs * n_billing_tokens_in_dataset} tokens")

dataset name = train ----------------------------------------------------------------------------------------------------
No errors found
Num examples missing system message: 0
Num examples missing user message: 0

#### Distribution of num_messages_per_example:
min / max: 3, 3
mean / median: 3.0, 3.0
p5 / p95: 3.0, 3.0

#### Distribution of num_total_tokens_per_example:
min / max: 50, 117
mean / median: 68.08, 56.0
p5 / p95: 50.0, 94.70000000000002

#### Distribution of num_assistant_tokens_per_example:
min / max: 7, 39
mean / median: 14.54, 10.5
p5 / p95: 7.0, 26.1

0 examples may be over the 4096 token limit, they will be truncated during fine-tuning
Dataset has ~3404 tokens that will be charged for during training
By default, you'll train for 3 epochs on this dataset
By default, you'll be charged for ~10212 tokens
dataset name = test ----------------------------------------------------------------------------------------------------
No errors found
Num examples missing system messag

In [159]:
def write_jsonl(data, filename):
    """
    Write a list of dictionaries to a JSON Lines file.

    Args:
    - data (list): List of dictionaries to write to the file.
    - filename (str): Name of the output JSON Lines file.

    Returns:
    - None
    """
    with open(filename, 'w') as file:
        for item in data:
            json.dump(item, file)
            file.write('\n')

current_time = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
train_jsonl_path = project_folder / f"train_data_{current_time}.jsonl"
validation_jsonl_path = project_folder / f"validation_data_{current_time}.jsonl"
test_jsonl_path = project_folder / f"test_data_{current_time}.jsonl"

write_jsonl(train, train_jsonl_path)
write_jsonl(valid, validation_jsonl_path)
write_jsonl(test, test_jsonl_path)
print("done.")
print("Wrote to", train_jsonl_path, "\n and", test_jsonl_path, "\n and", validation_jsonl_path)

done.
Wrote to /content/drive/My Drive/cs229-audio-project/fine_tuning_llm/train_data_2024-03-16_05-27-40.jsonl 
 and /content/drive/My Drive/cs229-audio-project/fine_tuning_llm/test_data_2024-03-16_05-27-40.jsonl 
 and /content/drive/My Drive/cs229-audio-project/fine_tuning_llm/validation_data_2024-03-16_05-27-40.jsonl


In [127]:
!pip install openai

In [57]:
from openai import OpenAI


In [139]:
# file = client.files.create(
#   file=open(train_jsonl_path, "rb"),
#   purpose="fine-tune"
# )

In [140]:
# print(file.id)
# client.fine_tuning.jobs.create(
#   training_file=file.id,
#   model="gpt-3.5-turbo"
# )

file-UX2OXldVtNzF8EffFgjbaavM


FineTuningJob(id='ftjob-IFDMxYSyvH3YRdgcA5wGiyZ6', created_at=1710565021, error=Error(code=None, message=None, param=None, error=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_epochs='auto', batch_size='auto', learning_rate_multiplier='auto'), model='gpt-3.5-turbo-0125', object='fine_tuning.job', organization_id='org-vt5vDvT6zTCqmBJzGnq1MVZM', result_files=[], status='validating_files', trained_tokens=None, training_file='file-UX2OXldVtNzF8EffFgjbaavM', validation_file=None, user_provided_suffix=None)

In [109]:
# finetune_job_file = file.id

In [160]:
client.fine_tuning.jobs.list()

SyncCursorPage[FineTuningJob](data=[FineTuningJob(id='ftjob-Aw7IazQGaVnx9pCIm18LY9ie', created_at=1710566954, error=Error(code=None, message=None, param=None, error=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_epochs=3, batch_size=1, learning_rate_multiplier=2), model='gpt-3.5-turbo-1106', object='fine_tuning.job', organization_id='org-vt5vDvT6zTCqmBJzGnq1MVZM', result_files=[], status='running', trained_tokens=None, training_file='file-1orpuySX09efOFwnq0T6VJiZ', validation_file='file-HOrfM5JKI3oxYAM9lB7xKcsL', user_provided_suffix='latexer'), FineTuningJob(id='ftjob-IFDMxYSyvH3YRdgcA5wGiyZ6', created_at=1710565021, error=Error(code=None, message=None, param=None, error=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_epochs=3, batch_size=1, learning_rate_multiplier=2), model='gpt-3.5-turbo-0125', object='fine_tuning.job', organization_id='org-vt5vDvT6zTCqmBJzGnq1MVZM', result_files=[], status='cancelled', trained_to

In [161]:
# import pprint

# def get_fine_tune_status(fdjob_str):
#     # List 10 fine-tuning jobs
#     pprint.pprint(client.fine_tuning.jobs.list(limit=10))

#     # Retrieve the state of a fine-tune
#     pprint.pprint(client.fine_tuning.jobs.retrieve(fdjob_str))

#     # # Cancel a job
#     # client.fine_tuning.jobs.cancel(fdjob_str)

#     # List up to 10 events from a fine-tuning job
#     pprint.pprint(client.fine_tuning.jobs.list_events(fine_tuning_job_id=fdjob_str, limit=10))

#     # # Delete a fine-tuned model (must be an owner of the org the model was created in)
#     # client.models.delete("ft:gpt-3.5-turbo:acemeco:suffix:abc123")

# get_fine_tune_status("ftjob-IFDMxYSyvH3YRdgcA5wGiyZ6") # is the fine tuning job id (printed in funciton above when listing)

FileObject(id='file-6zm6rc5nS37yq7LDu6x8j2rt', bytes=109486, created_at=1710564678, filename='train_data_2024-03-16_04-51-07.jsonl', object='file', purpose='fine-tune', status='processed', status_details=None)

In [162]:
def print_last_events(fdjob_str):
    response = client.fine_tuning.jobs.list_events(fdjob_str)

    events = response.data
    events.reverse()

    for event in events:
        # print(event)
        print(event.created_at, event.message, event)



print_last_events("ftjob-Aw7IazQGaVnx9pCIm18LY9ie")


1710566954 Created fine-tuning job: ftjob-Aw7IazQGaVnx9pCIm18LY9ie FineTuningJobEvent(id='ftevent-sGLDzAba6WmoyMkF9QIkgq7g', created_at=1710566954, level='info', message='Created fine-tuning job: ftjob-Aw7IazQGaVnx9pCIm18LY9ie', object='fine_tuning.job.event', data={}, type='message')
1710566954 Validating training file: file-1orpuySX09efOFwnq0T6VJiZ and validation file: file-HOrfM5JKI3oxYAM9lB7xKcsL FineTuningJobEvent(id='ftevent-5JbSTdtu9rkd4YoDvFNDsRmY', created_at=1710566954, level='info', message='Validating training file: file-1orpuySX09efOFwnq0T6VJiZ and validation file: file-HOrfM5JKI3oxYAM9lB7xKcsL', object='fine_tuning.job.event', data={}, type='message')
1710566976 Files validated, moving job to queued state FineTuningJobEvent(id='ftevent-Kw1g9uH7C3xdV65vo0vz76kD', created_at=1710566976, level='info', message='Files validated, moving job to queued state', object='fine_tuning.job.event', data={}, type='message')
1710566977 Fine-tuning job started FineTuningJobEvent(id='fteven